<div style="font-size:30px" align="center"> <b> Training Word2Vec on Biomedical Abstracts in PubMed </b> </div>

<div style="font-size:18px" align="center"> <b> Brandon Kramer - University of Virginia's Biocomplexity Institute </b> </div>

<br>

This notebook borrows from several resources to train a Word2Vec model on a subset of the PubMed database taken from January 2021. Overall, I am interested in testing whether diversity and racial terms are becoming more closely related over time. To do this, I train the model on 1990-1995 data and then a random sample of 2015-2020 data. 

#### Import packages and ingest data 

Let's load all of our packages 

In [4]:
# load packages
import os
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import Word
from gensim.models import Word2Vec
import multiprocessing 

# set cores, grab stop words
cores_available = multiprocessing.cpu_count() - 1
stop = stopwords.words('english')

#### Matching the Sample Sizes 

Since the 2010-2020 data is larger than the 1990-2000 data, we want to take a random sample of the later data to make the sample sizes the same for comparison later.

In [5]:
os.chdir("/sfs/qumulo/qhome/kb7hp/git/diversity/data/word_embeddings/")
pubmed_earlier = pd.read_csv("transformed_1990_2000_0821.csv")
pubmed_earlier['abstract_clean'] = pubmed_earlier['abstract'].str.lower()
pubmed_earlier['abstract_clean'] = pubmed_earlier['abstract_clean'].str.replace('-', ' ')
pubmed_earlier['abstract_clean'] = pubmed_earlier['abstract_clean'].str.replace(r'[^\w\s]+', '', regex=True)
pubmed_earlier['abstract_clean'] = pubmed_earlier['abstract_clean'].apply(lambda x:' '.join(x for x in x.split() if  not x.isdigit()))
pubmed_earlier['abstract_clean'] = pubmed_earlier['abstract_clean'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))
pubmed_earlier['abstract_clean'] = pubmed_earlier['abstract_clean'].apply(lambda x:' '.join([Word(word).lemmatize() for word in x.split()]))
pubmed_earlier.head()

,fk_pmid,year,abstract,abstract_clean
0,1366494,1990,although the discovery and transfer of technol...,although discovery transfer technology univers...
1,1366495,1990,strategies for the development of diagnostic p...,strategy development diagnostic product acquir...
2,1366496,1990,contemporary vaccines are relying increasingly...,contemporary vaccine relying increasingly mode...
3,1366497,1990,pseudomonas oleovorans can grow on linear alka...,pseudomonas oleovorans grow linear alkane alke...
4,1366498,1990,there are many naturally occurring adhesive pr...,many naturally occurring adhesive protein pote...


#### Cleaning the text data

Convert all text to lower case, remove punctuation, numbers, dots, digits and stop words, and finally lemmatize. 

In [6]:
os.chdir("/sfs/qumulo/qhome/kb7hp/git/diversity/data/word_embeddings/")
pubmed_later = pd.read_csv("transformed_2010_2020_0821.csv")
pubmed_later = pubmed_later[pubmed_later['abstract'].notnull()]
pubmed_later['abstract_clean'] = pubmed_later['abstract'].str.lower()
pubmed_later['abstract_clean'] = pubmed_later['abstract_clean'].str.replace('-', ' ')
pubmed_later['abstract_clean'] = pubmed_later['abstract_clean'].str.replace(r'[^\w\s]+', '', regex=True)
pubmed_later['abstract_clean'] = pubmed_later['abstract_clean'].apply(lambda x:' '.join(x for x in x.split() if not x.isdigit()))
pubmed_later['abstract_clean'] = pubmed_later['abstract_clean'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))
pubmed_later['abstract_clean'] = pubmed_later['abstract_clean'].apply(lambda x:' '.join([Word(word).lemmatize() for word in x.split()]))
pubmed_later.head()
pubmed_later.count()

fk_pmid           1131646
year              1131646
abstract          1131646
abstract_clean    1131646
dtype: int64

#### Training the Word2Vec Models 

Now, let's train these Word2Vec models and save them as a binary file to visualize later. 

In [ ]:
# run the model on the earlier data 
earlier_list=[]
for i in pubmed_earlier['abstract_clean']:
    li = list(i.split(" "))
    earlier_list.append(li)
earlier_model = Word2Vec(earlier_list, min_count=5, vector_size=512, window=5, epochs=5, seed=123, workers=cores_available)

os.chdir("/sfs/qumulo/qhome/kb7hp/git/diversity/data/word_embeddings/")
earlier_model.save("word2vec_1990_2000_socdiv_0821.model")
earlier_model.save("word2vec_1990_2000_socdiv_0821.bin")

# run the model on the later data 
later_list=[]
for i in pubmed_later['abstract_clean']:
    li = list(i.split(" "))
    later_list.append(li)
later_model = Word2Vec(later_list, min_count=5, vector_size=512, window=5, epochs=5, seed=123, workers=cores_available)

os.chdir("/sfs/qumulo/qhome/kb7hp/git/diversity/data/word_embeddings/")
later_model.save("word2vec_2010_2020_socdiv_0821.model")
later_model.save("word2vec_2010_2020_socdiv_0821.bin")

#### References 

[Guru 99's Tutorial on Word Embeddings](https://www.guru99.com/word-embedding-word2vec.html)

[Stackoverflow Post on Lemmatizing in Pandas](https://stackoverflow.com/questions/47557563/lemmatization-of-all-pandas-cells)